In [1]:
import pandas as pd
import re
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
import copy
from collections import defaultdict

In [2]:
e_topics = pd.read_csv("e_topic.csv.zip")
s_topics = pd.read_csv("s_topic.csv.zip")
g_topics = pd.read_csv("g_topic.csv.zip")
e_topics.head()

,topic,word,type
0,Экология,аудит системы,1
1,Экология,взаимодействие,1
2,Экология,внедрение,1
3,Экология,воздействие,1
4,Экология,восстановительные мероприятия,1


In [3]:
df = pd.concat([e_topics, s_topics, g_topics])
df["topic"].value_counts()

Отчетность и прозрачность             530
Лидерство                             371
Инвестиции и капитальные вложения     370
Безопасность и охрана труда           349
Эффективность и производительность    319
Трудовые отношения                    313
Дивиденды и акционеры                 306
Благотворительность                   259
Управление рисками                    214
Оплата труда                          131
Климат                                119
Инновации                             104
Отношения с потребителями              95
Профсоюзы и коллективные договоры      93
Предотвращение коррупции               92
Отношения с работниками                84
Налоги                                 81
Вода                                   69
Энергия                                63
Экология                               59
Отходы                                 51
Обучение и развитие                    50
Безопасность продукта                  50
Отношения  потребителями          

In [4]:
re_dict = {}
sheets_dict = {}
base_dict = {}

for idx, row in df.iterrows():
    if not sheets_dict.get(row["topic"], False):
        sheets_dict[row["topic"]] = {}
        base_dict[row["topic"]] = {}
    sheets_dict[row["topic"]][row["word"]] = row["type"]
    base_dict[row["topic"]][row["word"]] = 0

In [5]:
for key, val in sheets_dict.items():
    re_dict[key] = re.compile("(" + "(?!\w)|".join(val.keys()) + "(?!\w))")

In [6]:
text = "социальный отчёт полностью охватывать социальный политика оао ммк дальнейший расширение возможно часть увеличение показатель деятельность случай выявление заинтересованность пользователь информация общество"

for key, regex in re_dict.items():
    print(key, regex.findall(text))

Экология []
Климат ['политика']
Энергия []
Воздух []
Вода []
Отходы []
Биоразнообразие []
Социально-культурное воздействи []
Антикоррупция ['политика']
Дивиденды и акционеры []
Инновации []
Лидерство ['политика', 'показатель']
Отчетность и прозрачность ['политика']
Предотвращение коррупции ['политика']
Управление рисками ['ность', 'ность']
Эффективность и производительность ['показатель', 'ция']
Безопасность и охрана труда ['политика', 'информация']
Безопасность продукта []
Благотворительность []
Инвестиции и капитальные вложения ['политика']
Налоги []
Обучение и развитие []
Оплата труда []
Отношения  потребителями []
Отношения с потребителями []
Отношения с работниками []
Охрана здоровья []
Профсоюзы и коллективные договоры []
Трудовые отношения ['показатель']


In [7]:
df = pd.read_csv("paragraphs.csv.zip") # , usecols=["cleaned_text"]
df = df.dropna()
# df = df[df["cleaned_text"].apply(lambda x: len(x.split()) > 1)]
# df = df.reset_index(drop=True)
df.shape

<ipython-input-7-f512ae0502ff>:1: DtypeWarning: Columns (0,1,2,3,4,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("paragraphs.csv.zip") # , usecols=["cleaned_text"]


(2309485, 8)

In [11]:
topics = []
for text in tqdm(df["cleaned_text"].tolist()):
    paragraph_dict = copy.deepcopy(base_dict)
    found_words = False
    for key, regex in re_dict.items():
        words = regex.findall(text)
        for word in words:
            paragraph_dict[key][word] = 1
            found_words = True
    res = {}
    if not found_words:
        res = {key: 0 for key in sheets_dict}
        topics.append({**res, **{"max_score": None, "max_topic": None}})
        continue
    for key, vector in sheets_dict.items():
        sim = cosine_similarity(X=[list(vector.values())], Y=[list(paragraph_dict[key].values())])[0][0]
        res[key] = sim
    
    topics.append({**res, **{"max_score": max(res.values()), "max_topic": max(res, key=lambda k: res[k])}})

100%|██████████| 2309485/2309485 [9:17:55<00:00, 68.99it/s]   


In [12]:
import pickle

with open("topics.pkl", "wb") as fp:   #Pickling
    pickle.dump(topics, fp)

In [13]:
topics = pd.DataFrame(topics)
topics.to_csv("all_topics.csv.zip")

In [14]:
df = df.join(topics)
df.to_csv("paragrapghs_with_topisc.csv.zip")

In [17]:
df["max_topic"].value_counts()

Антикоррупция                         39950
Управление рисками                    38209
Безопасность продукта                 33122
Отношения с работниками               32046
Отношения с потребителями             30135
Экология                              29904
Налоги                                29683
Дивиденды и акционеры                 23461
Лидерство                             20550
Эффективность и производительность    18737
Обучение и развитие                   17817
Отношения  потребителями              16615
Предотвращение коррупции              14611
Энергия                               13266
Отчетность и прозрачность             12888
Климат                                12041
Инвестиции и капитальные вложения     12027
Профсоюзы и коллективные договоры     10916
Благотворительность                   10653
Инновации                              9400
Трудовые отношения                     9082
Отходы                                 8627
Вода                            

In [44]:
grouped = pd.DataFrame(df.groupby(["company", "year", "max_topic"])["cleaned_text"].agg(' '.join))
grouped.head()

cleaned_text
company     year max_topic                                                                     
Alcoa, Inc. 2004 Антикоррупция                москва белый калитва восток московский офис са...
                 Безопасность и охрана труда          доход европа процент общий доход компания
                 Безопасность продукта        я очень приятно представить ваш внимание первы...
                 Благотворительность          лос летучий органический соединение в тысяча м...
                 Налоги                       возрастание выброс происходить связь применени...

In [45]:
from collections import defaultdict

topics_df = pd.concat([e_topics, s_topics, g_topics])
len_topics = defaultdict(dict)

for idx, row in tqdm(topics_df.iterrows(), total=topics_df.shape[0]):
    if not len_topics[row["topic"]].get(row["type"], False):
        len_topics[row["topic"]][row["type"]] = 0
    len_topics[row["topic"]][row["type"]] += 1

100%|██████████| 4414/4414 [00:00<00:00, 19458.61it/s]


In [46]:
len_topics

defaultdict(dict,
            {'Экология': {1: 51, 2: 8},
             'Климат': {1: 95, 2: 24},
             'Энергия': {1: 52, 2: 11},
             'Воздух': {1: 42},
             'Вода': {1: 61, 2: 8},
             'Отходы': {1: 41, 2: 10},
             'Биоразнообразие': {1: 47},
             'Социально-культурное воздействи': {1: 3},
             'Антикоррупция': {1: 50},
             'Дивиденды и акционеры': {1: 306},
             'Инновации': {1: 104},
             'Лидерство': {1: 371},
             'Отчетность и прозрачность': {1: 530},
             'Предотвращение коррупции': {1: 92},
             'Управление рисками': {1: 214},
             'Эффективность и производительность': {1: 319},
             'Безопасность и охрана труда': {1: 349},
             'Безопасность продукта': {1: 50},
             'Благотворительность': {1: 259},
             'Инвестиции и капитальные вложения': {1: 370},
             'Налоги': {1: 81},
             'Обучение и развитие': {1: 50},
        

In [47]:
from __future__ import annotations

def get_score(words: dict[str, int], topic: str) -> int:
    count_words = {key: 0 for key in len_topics[topic]}
    for word in words:
        count_words[sheets_dict[topic][word]] += 1
    base_percent = count_words[1] / len_topics[topic][1]
    advanced_percent =  count_words[2] / len_topics[topic][2] if count_words.get(2, False) else 0
    if base_percent < 0.20:
        return 1
    elif base_percent < 0.40:
        return 2
    elif base_percent < 0.60:
        return 3
    elif base_percent < 0.80:
        return 4
    elif base_percent <= 1 and advanced_percent == 0:
        return 5
    elif base_percent <= 1 and advanced_percent < 0.40:
        return 6
    elif base_percent <= 1 and advanced_percent < 0.80:
        return 7
    elif base_percent <= 1 and advanced_percent <= 1:
        return 7
    else:
        return 0

In [55]:
scores = []

for (company, year, topic), row in tqdm(grouped.iterrows(), total=grouped.shape[0]):
    paragraph_dict = copy.deepcopy(base_dict)[topic]
    text = row["cleaned_text"]
    words = re_dict[topic].findall(text)
    for word in words:
        paragraph_dict[word] = 1
    scores.append(get_score(paragraph_dict, topic))

100%|██████████| 8788/8788 [01:05<00:00, 135.19it/s]


In [57]:
grouped["score"] = scores
grouped.head(20)

cleaned_text  \
company     year       max_topic                                                                              
Alcoa, Inc. 2004       Антикоррупция                      москва белый калитва восток московский офис са...   
                       Безопасность и охрана труда                доход европа процент общий доход компания   
                       Безопасность продукта              я очень приятно представить ваш внимание первы...   
                       Благотворительность                лос летучий органический соединение в тысяча м...   
                       Налоги                             возрастание выброс происходить связь применени...   
                       Отношения с потребителями                 белый калитва любучан восток москва самара   
                       Отношения с работниками            вступление выброс загрязнять вещество атмосфер...   
                       Отходы                             показатель здоровье работник безопасность труд...   
                       Предотвращение коррупции           наш преимущество заключаться производить очень...   
                       Профсоюзы и коллективные договоры  показатель экологический безопасность для комп...   
                       Управление рисками                 отчёт устойчивый развитие год отчёт ход выполн...   
BIOCAD      2019, 2018 Антикоррупция                      соответствие отчёт руководство прямой речь пре...   
                       Безопасность и охрана труда        экопроект павел яко вл быть в ранний разработк...   
                       Безопасность продукта              содержание устойчивый развитие стратегия приор...   
                       Благотворительность                август россия мир уровень вовлечённость сотруд...   
                       Вода                               характер собственность форма существенный изме...   
                       Дивиденды и акционеры              сокращать отход производство упаковка снижать ...   
                       Инвестиции и капитальные вложения  самый продавать препарат одержать победа суд п...   
                       Инновации                          генферон® ацеллбия® альгерон® эфлейра® млн руб...   
                       Климат                             о компания отход удаться ускорить синтезирован...   

                                                          score  
company     year       max_topic                                 
Alcoa, Inc. 2004       Антикоррупция                          5  
                       Безопасность и охрана труда            5  
                       Безопасность продукта                  5  
                       Благотворительность                    5  
                       Налоги                                 5  
                       Отношения с потребителями              5  
                       Отношения с работниками                5  
                       Отходы                                 7  
                       Предотвращение коррупции               5  
                       Профсоюзы и коллективные договоры      5  
                       Управление рисками                     5  
BIOCAD      2019, 2018 Антикоррупция                          5  
                       Безопасность и охрана труда            5  
                       Безопасность продукта                  5  
                       Благотворительность                    5  
                       Вода                                   7  
                       Дивиденды и акционеры                  5  
                       Инвестиции и капитальные вложения      5  
                       Инновации                              5  
                       Климат                                 7

In [58]:
grouped.to_csv("grouped_paragraphs.csv.zip")